In [1]:
# note: glove runs in cpu
# download glove code and compile
# the exclamation mark is for running linux commands in colab
!git clone http://github.com/stanfordnlp/glove
! cd glove && make


Cloning into 'glove'...
remote: Enumerating objects: 592, done.
remote: Total 592 (delta 0), reused 0 (delta 0), pack-reused 592
Receiving objects: 100% (592/592), 220.54 KiB | 5.65 MiB/s, done.
Resolving deltas: 100% (338/338), done.
mkdir -p build
gcc -c src/vocab_count.c -o build/vocab_count.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
gcc -c src/cooccur.c -o build/cooccur.o -lm -pthread -O3 -march=native -funroll-loops -Wall -Wextra -Wpedantic
src/cooccur.c: In function ‘merge_files’:
src/cooccur.c:180:9: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
         fread(&new, sizeof(CREC), 1, fid[i]);
         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:190:5: warning: ignoring return value of ‘fread’, declared with attribute warn_unused_result [-Wunused-result]
     fread(&new, sizeof(CREC), 1, fid[i]);
     ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
src/cooccur.c:203:9: warning: ignoring return valu

In [2]:
#download csv files from gdrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
# ask google for authorization 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
#download file from gdrive with the id for share file
# create share link for tar.gz file  and copy id
#https://drive.google.com/file/d/1E279yz7ZiZmok6qOYWlrWku1w7F13T5Q/view?usp=sharing
file_id = '1E279yz7ZiZmok6qOYWlrWku1w7F13T5Q' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_text_data_2.tar.gz')

In [4]:
#extract data
!tar -xzf test_text_data_2.tar.gz

In [6]:
#upload mimic_utils_text.py
from google.colab import files
uploaded = files.upload()

Saving mimic_utils_text.py to mimic_utils_text.py


In [7]:
# dump text notes from the training split of the data into a text file
from __future__ import absolute_import
import numpy as np
import sys
import codecs

#TODO
from mimic_utils_text import InHospitalMortalityReader

reader = InHospitalMortalityReader(dataset_dir='test_text_data_2/in-hospital-mortality/train',
                              notes_dir='test_text_data_2/train',  
                              listfile='test_text_data_2/in-hospital-mortality/train/listfile.csv')

# over all patients in training
N = reader.get_number_of_examples()
with codecs.open("train_mimic.txt", "w", "utf-8") as textfile:
  for i in range(N):
      patient = reader.read_example(i)
      # get notes and print sentences into file
      patient_notes = patient['text']
      for doc, sentences in patient_notes.items():
          for s in sentences:
              print(' '.join(s), file=textfile)

In [8]:
# move text file into glove directory
!mv train_mimic.txt glove/train_mimic.txt

In [9]:
# download mimic_trainGlove.sh
# this script are the set of commands to run glove
from google.colab import files
uploaded = files.upload()

Saving mimic_trainGlove.sh to mimic_trainGlove.sh


In [11]:
# move mimic_trainGlove.sh to glove directory
!mv mimic_trainGlove.sh glove/mimic_trainGlove.sh

In [13]:
# use sh script to run 
# these commands train glove
# 
!cd glove && bash ./mimic_trainGlove.sh


$ build/vocab_count -min-count 5 -verbose 2 < train_mimic.txt > vocab.txt
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 to

In [15]:
# check the first word embeddings
!head glove/mimic_vectors_training.100d.txt

: 1.190956 -0.047710 -0.963017 1.415249 -0.206819 -0.383004 -0.180624 0.169717 -2.063898 -0.235796 -0.494100 -0.306141 2.030720 -0.967545 1.319234 -0.078401 0.007754 0.655695 -1.372468 -0.567435 0.459234 -0.379192 0.195114 1.567850 -0.017347 0.277815 -0.996987 0.519143 1.403502 0.377793 -0.163458 0.195770 -0.695664 0.219544 0.748933 -1.463034 1.300978 -1.090082 0.619144 0.560326 0.782781 -1.306317 0.436708 0.479997 -0.721851 -1.028805 -1.450909 2.268183 1.541740 -0.879070 -0.538632 -0.826458 -0.128308 0.515365 0.744273 -0.895940 0.529998 0.294070 -1.375170 -0.302488 -0.230789 1.542795 -0.419611 2.222409 0.496992 0.167534 -0.126733 -0.239394 -1.205657 -0.041976 0.052340 -0.610138 -0.420435 1.361392 1.531809 -0.086023 -1.085739 0.498637 0.454590 -0.374867 0.084343 0.189525 0.912582 0.005024 1.852522 -0.409155 -0.568926 1.038772 -1.097564 -0.830471 0.033013 -0.208007 0.142582 0.639180 0.455660 -0.339154 -0.549129 0.450661 0.166742 0.819326
, 1.366800 0.477397 0.883570 0.769593 -0.013484 0

In [ ]:
# remember to download your pre-trained word embeddings 
# to your hard drive (from files on the left: glove/mimic_vectors_training.100d.txt) or your Gdrive